# Create necessary classes

## General classes

In [ ]:
# package(s) related to the simulation
import simpy

# package(s) related to time, space and id
import datetime
import json
import geojson
import shapely.geometry
import uuid

In [ ]:
class Identifiable(object):
    """Something that has a name and id
    
    env: a simpy Environment
    name: a name
    id: a unique id generated with uuid"""
    
    def __init__(self, env, name, id=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.env = env
        self.name = name
        # generate some id, in this case based on m
        self.id = id if id else str(uuid.uuid1())
                

In [ ]:
class Location(object):
    """Something with a geometry (geojson format)
    
    geometry: can be a point as well as a polygon"""
    
    def __init__(self, geometry, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.geometry = geometry


In [ ]:
class Container(object):
    """Container class
    
    capacity: amount the container can hold
    level: amount the container holds
    container: a simpy object that can hold stuff
    total_reguested: a counter needed to prevent over-handling"""

    def __init__(self, capacity, level, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.container = simpy.Container(capacity, init=level)
        self.total_requested = 0


In [ ]:
class Move(object):
    """Move class
    
    origin: origin of trip
    destination: destination of trip
    v_empty: speed empty [m/s]
    v_full: speed full [m/s]"""
    
    def __init__(self, 
                 origin=None, destination=None,
                 v_empty=1.5, v_full=1, 
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.origin = origin
        self.destination = destination
        self.v_empty = v_empty 
        self.v_full = v_full 

    def execute(self):
        print("Setting sail from %s to %s" % (self.origin.geometry, self.destination.geometry))

        origin = shapely.geometry.asShape(self.origin.geometry)
        destination = shapely.geometry.asShape(self.destinaton.geometry)
        distance = origin.distance(destination) 

        if self.container.level == self.container.capacity:
            self.log_entry('transport full start', self.env.now, self.container.level)
            yield self.env.timeout(distance / self.v_full)
            self.log_entry('transport full stop', self.env.now, self.container.level)

        elif self.container.level == 0:
            self.log_entry('transport empty start', self.env.now, self.container.level)
            yield self.env.timeout(distance / self.v_empty)
            self.log_entry('transport empty stop', self.env.now, self.container.level)
                

In [ ]:
class Process(object):
    """Process class
    
    get_obj: simpy Container from which to extract (can be Site or Vessel)
    put_obj: simpy Container in which to insert (can be Site or Vessel)
    p_rate: rate with which quantity can be processed
    """

    def __init__(self, 
                 origin, destination, rate, amount=0, 
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.origin=origin
        self.destination=destination
        self.rate=rate
        self.amount=amount
        
    def execute(self):
        self.log_entry('process start', self.env.now, self.container.level)
        yield self.env.timeout(distance / self.v_empty)
        self.log_entry('process stop', self.env.now, self.container.level)
        

In [ ]:
class Log(object):
    """Log class
    
    log: log message [format: 'start activity' or 'stop activity']
    t: timestamp
    value: a value can be logged as well"""
        
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.log = []
        self.t = []
        self.value = []
        
    def log_entry(self, log, t, value):
        """Log"""
        self.log.append(log)
        self.t.append(t)
        self.value.append(value)
        

## Constructed classes

In [ ]:
class Site(Identifiable, Location, Log, Container):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [ ]:
class Transport_Resource(Identifiable, Location, Log, Container, Move):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [ ]:
class Transport_Processing_Resource(Identifiable, Location, Log, Container, Move, Process):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [ ]:
class Processing_Resource(Identifiable, Location, Log, Process):
    def __init__(self, 
                 *args, **kwargs):
        super().__init__(*args, **kwargs)


## Activities

In [ ]:
class Installation(Identifiable):
    """The Installation Class forms a spefic class of activities with associated methods that can 
    initiate and suspend processes according to a number of specified conditions. This class deals 
    mainly with transport and installation of discrete objects."""

    def __init__(self, 
                 condition,
                 origin, destination,  
                 loader, mover, unloader,
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        
        self.condition = condition
        
        self.origin = origin
        self.destination = destination
        
        self.loader = loader
        self.mover = mover
        self.unloader = unloader
        
        self.standing_by_proc = env.process(
            self.standing_by(env, 
                             condition,
                             origin, destination,
                             loader, mover, unloader))

        self.installation_proc = env.process(
            self.installation_process_control(env,
                             condition,
                             origin, destination,
                             loader, mover, unloader))

        self.installation_reactivate = env.event()

    def standing_by(self, env, condition,
                          origin, destination,
                          loader, mover, unloader):
        """Standing by"""
        shown = False

        while not eval(condition):
            if not shown:
                print('T=' + '{:06.2f}'.format(env.now) + ' ' + self.name + ' to ' + destination.name + ' suspended')
                shown = True
            yield env.timeout(1) # step 1 minute ahead

        print('T=' + '{:06.2f}'.format(env.now) + ' ' + 'Condition: ' + condition + ' is satisfied')

        self.installation_reactivate.succeed()  # "reactivate"
        self.installation_reactivate = env.event()
 
    def installation_process_control(self, env, condition,
                                           origin, destination,
                                           loader, mover, unloader):
        """Installation process control"""  
        while not eval(condition):
            yield self.installation_reactivate

        print('T=' + '{:06.2f}'.format(env.now) + ' '+ self.name + ' to ' + destination.name + ' started')
        while eval(condition):
            yield from self.installation_process(env, condition,
                                                 origin, destination,
                                                 loader, mover, unloader)
        
    def installation_process(self, env, condition,
                                   origin, destination,
                                   loader, mover, unloader):
        """Installation process"""
        amount = min(
            origin.container.level,
            mover.container.capacity - mover.container.level,
            destination.container.capacity - destination.container.level)
        print(amount)
        if amount>0:
            # request access to the load_resource
            with loader.resource.request() as my_load_resource_turn:
                yield my_load_resource_turn

                destination.total_requested += amount
                origin.total_requested += amount

                yield from loader.Process.execute(origin, mover, amount)

                loader.resource.release(my_load_resource_turn)

            # request access to the transport_resource
            with mover.resource.request() as my_mover_turn:
                yield my_mover_turn

                yield from mover.Move.execute()

            # request access to the placement_resource
            with unloader.resource.request() as my_unloader_turn:
                yield my_unloader_turn

                yield from loader.load(mover, destination, amount)

                unloader.resource.release(my_unloader_turn)

            # request access to the transport_resource
            with mover.resource.request() as my_mover_turn:
                yield my_mover_turn

                yield from mover.Move.execute()

                mover.resource.release(my_mover_turn)


# Start case

In [ ]:
import simpy

In [ ]:
# *** Create a project environment
env = simpy.Environment()

In [ ]:
# *** Define sites

# Den Oever
# <longitude>5.070195628786471</longitude>
# <latitude>52.93917167503315</latitude>

# Breezanddijk
# <longitude>5.197016484858931</longitude>
# <latitude>53.0229621352376</latitude>

# Kornwerderzand
# <longitude>5.310252030674185</longitude>
# <latitude>53.06330503232375</latitude>

# Den Oever haven
# <longitude>5.019298185633251</longitude>
# <latitude>52.94239823421129</latitude>

# sites in database (nr indicates km's from Den Oever haven)
data_site_00 = {"env": env,
                "name": "KP00", "geometry": geojson.Point([52.94239823421129, 5.019298185633251]),
                "capacity": 15000, "level": 0}
data_site_01 = {"env": env,
                "name": "KP01", "geometry": geojson.Point([52.93917167503315, 5.070195628786471]),
                "capacity": 5000, "level": 0}
data_site_15 = {"env": env,
                "name": "KP15", "geometry": geojson.Point([53.0229621352376,  5.197016484858931]),
                "capacity": 5000, "level": 0}
data_site_32 = {"env": env,
                "name": "KP32", "geometry": geojson.Point([53.06330503232375, 5.310252030674185]),
                "capacity": 5000, "level": 0}

# create site objects
site_00 = Site(**data_site_00)
site_01 = Site(**data_site_01)
site_15 = Site(**data_site_15)
site_32 = Site(**data_site_32)

# print the outputs
print(site_00.__dict__)
print(site_01.__dict__)
print(site_15.__dict__)
print(site_32.__dict__)

In [ ]:
# *** Define fleet

# sites in database (nr indicates km's from Den Oever haven)
# - processing resources
data_gantry_crane = {"env": env,
                "name": "Gantry crane", "geometry": geojson.Point([52.94239823421129, 5.019298185633251]),
                "origin": site_00, "destination": site_01, "rate": 0.10}
data_installation_crane = {"env": env,
                "name": "Installation crane", "geometry": geojson.Point([53.0229621352376,  5.197016484858931]),
                "origin": site_00, "destination": site_01, "rate": 0.05}

# - transport resources
data_transport_barge_01 = {"env": env,
                "name": "Transport barge 01", "geometry": geojson.Point([52.93917167503315, 5.070195628786471]),
                "capacity": 1000, "level": 0}
data_transport_barge_02 = {"env": env,
                "name": "Transport barge 02", "geometry": geojson.Point([52.93917167503315, 5.070195628786471]),
                "capacity": 1000, "level": 0}

# create site objects
# - processing resources
gantry_crane = Processing_Resource(**data_gantry_crane)
installation_crane = Processing_Resource(**data_installation_crane)

# - transport resources
transport_barge_01 = Transport_Resource(**data_transport_barge_01)
transport_barge_02 = Transport_Resource(**data_transport_barge_02)

# print the outputs
print(gantry_crane.__dict__)
print(transport_barge_01.__dict__)
print(transport_barge_02.__dict__)
print(installation_crane.__dict__)


In [ ]:
# *** Define activties

# activities in database
data_act_1 = {"env": env,
              "name": "Block placement",
              "origin": site_00, "destination": site_01,
              "loader": gantry_crane, "mover": transport_barge_01, "unloader": installation_crane,
              "condition": '''site_01.container.level<5000'''}
data_act_2 = {"env": env,
              "name": "Block placement",
              "origin": site_00, "destination": site_15,
              "loader": gantry_crane, "mover": transport_barge_02, "unloader": installation_crane,
              "condition": '''site_15.container.level<5000 and site_01.container.level>=3000'''}

# create Activity objects
act_1 = Installation(**data_act_1)
act_2 = Installation(**data_act_2)

# print the outputs
print(act_1.__dict__)
print(act_2.__dict__)


In [ ]:
#*** Run the project
env.run()

# Mixins

We can implement the different aspects of our components as mixins. Make sure the different aspects are not dependent on each other. 

In [ ]:
# example of how to initialize different mixins
# The mixin pattern is a way to dynamically compose a type, based on so called mixins
import json
import geojson
import uuid

class Identifiable(object):
    """Something that has a name and id"""
    def __init__(self, name, id=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.name = name
        # generate some id, in this case based on m
        self.id = id if id else str(uuid.uuid1())
        print("Name:", self.name)

class Location(object):
    """Something with a geometry (geojson format)"""
    def __init__(self, geometry, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.geometry = geometry
        print("Location:", self.geometry)

class Transport(object):
    """Something that can move"""
    def __init__(self, env, from_location, to_location, speed, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.env = env
        self.speed = speed
        self.from_location = from_location
        self.to_location = to_location
        self.location = from_location
        print("Transport")
    def move(self):
        print("Setting sail from %s to %s" % (self.from_location.geometry, self.to_location.geometry))
        self.location = self.to_location
        to = shapely.geometry.asShape(self.to_location.geometry)
        from_ = shapely.geometry.asShape(self.from_location.geometry)
        distance = from_.distance(to) 
        yield self.env.timeout(distance / self.speed)

In [ ]:


class Site(Identifiable, Location):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        print("Site:", self.__dict__)
    

# data in database
data = {"name": "project", "geometry": geojson.Point([52, 3])}
data2 = {"name": "Den Oever", "geometry": geojson.Point([52.5, 4])}
print("we start with data", data)
        
# now you have a class with all the variables stored in the instance
print("now we load data into the Site")
site = Site(**data)

# Create a DynamicSite type in runtime based on the building blocks Location and Identifiable
print("now we create a DynamicSite, a custom type that you might want to create in a ui")
DynamicSite = type("DynamicSite", (Location, Identifiable), {})

env = simpy.Environment()

# now we can load data into it
print("we load data into the dynamic site")
dynamic_site_1 = DynamicSite(**data)
dynamic_site_2 = DynamicSite(**data2)
transport = Transport(env, from_location=dynamic_site_1, to_location=dynamic_site_2, speed=0.3)
env.process(transport.move())
# now we can serialize it
json.dumps(site.__dict__), json.dumps(dynamic_site_1.__dict__)
env.run()

In [ ]:
import shapely.geometry
import datetime
import dateutil

shape_1 = shapely.geometry.asShape(dynamic_site_1.geometry)
shape_2 = shapely.geometry.asShape(dynamic_site_2.geometry)
shape_1.distance(shape_2)


now = datetime.datetime.now()

now.timestamp() - datetime.datetime(1970, 1, 1, tzinfo=dateutil.tz.UTC).timestamp()

In [ ]:
import json

class EncodeNetwork(json.JSONEncoder):
    def default(self, *args, **kwargs):
        return super()


# Test space

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph()

In [ ]:
FG = nx.Graph()
FG.add_weighted_edges_from([(1, 2, 0.125), (1, 3, 0.75), (2, 4, 1.2), (3, 4, 0.375)])
for n, nbrs in FG.adj.items():
    for nbr, eattr in nbrs.items():
        wt = eattr['weight']
        if wt < 0.5: print('(%d, %d, %.3f)' % (n, nbr, wt))

In [ ]:
import json
transport_resource_1.__dict__